# Starting to explore daily train data

In [11]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import time
from dateutil.relativedelta import relativedelta

# Hack for using import with relative path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fetchTrainData import fetch_train_data
from fetchWeatherData import fetch_weather_data, save_weather_data, open_weather_data

weather_data = {}

def get_weather(stop_):
    tag = stop_['station.shortCode']
    if type(tag) == str:
        lat = stop_['station.location'][0]
        lon = stop_['station.location'][1]
        if tag not in weather_data:
            print("FETCHING DATA FOR: " + tag)
            weather_data[tag] = fetch_weather_data(unix_time, lat, lon)['list'][0]['main']['temp']

            
def extract_time_tables(train_stops_):
    print("TRAIN NUMBER " + str(train_stops_.name) + ", " + str(float(train_stops_.name) / number_of_trains))
    train_stops = pd.json_normalize(train_stops_)
    to_return = train_stops['differenceInMinutes'].sum()
    train_stops = train_stops.drop(['differenceInMinutes'], axis=1)

    train_stops.apply(get_weather, axis=1)

    for stop in train_stops['station.shortCode'].unique():
        if type(stop) == str:
            if stop not in df.columns:
                df[stop] = np.nan

            df[stop][train_stops_.name] = weather_data[stop]
        
    
    return to_return


loops = 12 # How many months data we want in the dataframe, apparently we can't get over 12 months data
number_of_trains = 750 # How many trains to fetch per month. Seems to be softlocked at about 1000, 500 is safe
all_data_frames = []
all_late_amounts = []


for i in range(loops):
    date = datetime.date.today() - relativedelta(months = i)
    unix_time = time.mktime(date.timetuple())
    print("LOOP " + str(i) + ", DATE: " + str(date))

    q = (
        """ 
        {
        """
          f'  trainsByDepartureDate(departureDate: \"{date}\",'
        """
        """
          f'  take: {number_of_trains},'
        """
            where: {
                and: [
                    {or: [{deleted: {unequals: null}}, 
                    {deleted: {equals: false}}]}, 
                    {cancelled: {equals: false}}, 
                    {operator: {shortCode: {equals: "vr"}}}
                    ]  
            }
          ) 
          {
            trainNumber
            departureDate
            timeTableRows {
            differenceInMinutes
              station {
                shortCode
                location
              }
            }
          }
        }
        """
        )

    result = fetch_train_data(q)
    recs = result['data']['trainsByDepartureDate']

    df = pd.json_normalize(recs)
    all_stops = pd.json_normalize(df['timeTableRows'])
    df = df.drop(['timeTableRows'], axis=1)

    weather_data = open_weather_data(date)
    how_much_late = all_stops.apply(extract_time_tables, axis=1)
    save_weather_data(date, weather_data)
    
    df['departureDate'] = date.month
    
    all_data_frames.append(df)
    all_late_amounts.append(how_much_late)
    
combined_train_data = pd.concat(all_data_frames).fillna(0).reset_index().drop(['index'], axis=1)
combined_late_amounts = pd.concat(all_late_amounts).fillna(0).reset_index().drop(['index'], axis=1)

combined_late_amounts.rename(columns={0: "total_late"}, inplace=True)
combined_late_amounts = combined_late_amounts["total_late"].apply(lambda x: int(x > 4))

print(combined_train_data)
print(combined_late_amounts)

LOOP 0, DATE: 2021-10-26
TRAIN NUMBER 0, 0.0
FETCHING DATA FOR: HKI
FETCHING DATA FOR: PSL
FETCHING DATA FOR: KÄP
FETCHING DATA FOR: OLK
FETCHING DATA FOR: PMK
FETCHING DATA FOR: ML
FETCHING DATA FOR: TNA
FETCHING DATA FOR: PLA
FETCHING DATA FOR: TKL
FETCHING DATA FOR: HKH
FETCHING DATA FOR: HVK
FETCHING DATA FOR: KVY
FETCHING DATA FOR: RKL
FETCHING DATA FOR: HNA
FETCHING DATA FOR: KRS
FETCHING DATA FOR: SAV
FETCHING DATA FOR: KE
FETCHING DATA FOR: KYT
FETCHING DATA FOR: HAA
FETCHING DATA FOR: KSU
FETCHING DATA FOR: MLÄ
FETCHING DATA FOR: SIP
FETCHING DATA FOR: HNN
FETCHING DATA FOR: HLT
FETCHING DATA FOR: LH
FETCHING DATA FOR: VLH
FETCHING DATA FOR: NSL
FETCHING DATA FOR: UKÄ
FETCHING DATA FOR: MKA
FETCHING DATA FOR: KA
FETCHING DATA FOR: NMÄ
FETCHING DATA FOR: SMÄ
FETCHING DATA FOR: KRA
FETCHING DATA FOR: KV
FETCHING DATA FOR: KUV
FETCHING DATA FOR: UTI
FETCHING DATA FOR: KTH
FETCHING DATA FOR: KPA
FETCHING DATA FOR: KJR
FETCHING DATA FOR: TA
FETCHING DATA FOR: LÄ
FETCHING DATA FOR: 

/tmp/ipykernel_25422/3181574030.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[stop] = np.nan


TRAIN NUMBER 13, 0.017333333333333333
TRAIN NUMBER 14, 0.018666666666666668
FETCHING DATA FOR: ROI
FETCHING DATA FOR: MUL
FETCHING DATA FOR: KVU
FETCHING DATA FOR: TRV
FETCHING DATA FOR: TÖR
FETCHING DATA FOR: LLA
FETCHING DATA FOR: LI
FETCHING DATA FOR: KEM
FETCHING DATA FOR: SIM
FETCHING DATA FOR: MYS
FETCHING DATA FOR: II
FETCHING DATA FOR: HD
FETCHING DATA FOR: TUA
TRAIN NUMBER 15, 0.02
TRAIN NUMBER 16, 0.021333333333333333
TRAIN NUMBER 17, 0.02266666666666667
TRAIN NUMBER 18, 0.024
TRAIN NUMBER 19, 0.025333333333333333
TRAIN NUMBER 20, 0.02666666666666667
TRAIN NUMBER 21, 0.028
TRAIN NUMBER 22, 0.029333333333333333
TRAIN NUMBER 23, 0.030666666666666665
TRAIN NUMBER 24, 0.032
TRAIN NUMBER 25, 0.03333333333333333
TRAIN NUMBER 26, 0.034666666666666665
FETCHING DATA FOR: VS
FETCHING DATA FOR: LAI
FETCHING DATA FOR: TK
FETCHING DATA FOR: IKY
FETCHING DATA FOR: YST
TRAIN NUMBER 27, 0.036
TRAIN NUMBER 28, 0.037333333333333336
TRAIN NUMBER 29, 0.03866666666666667
TRAIN NUMBER 30, 0.04
TRA

FETCHING DATA FOR: SKÄ
TRAIN NUMBER 153, 0.204
TRAIN NUMBER 154, 0.20533333333333334
TRAIN NUMBER 155, 0.20666666666666667
TRAIN NUMBER 156, 0.208
FETCHING DATA FOR: PUN
FETCHING DATA FOR: LUS
FETCHING DATA FOR: REE
FETCHING DATA FOR: KIÄ
FETCHING DATA FOR: PKY
FETCHING DATA FOR: SL
TRAIN NUMBER 157, 0.20933333333333334
TRAIN NUMBER 158, 0.21066666666666667
TRAIN NUMBER 159, 0.212
TRAIN NUMBER 160, 0.21333333333333335
TRAIN NUMBER 161, 0.21466666666666667
TRAIN NUMBER 162, 0.216
TRAIN NUMBER 163, 0.21733333333333332
TRAIN NUMBER 164, 0.21866666666666668
TRAIN NUMBER 165, 0.22
TRAIN NUMBER 166, 0.22133333333333333
TRAIN NUMBER 167, 0.22266666666666668
TRAIN NUMBER 168, 0.224
FETCHING DATA FOR: NRM
FETCHING DATA FOR: KOH
FETCHING DATA FOR: HÖL
FETCHING DATA FOR: VK
FETCHING DATA FOR: KYN
FETCHING DATA FOR: LIS
FETCHING DATA FOR: VSL
FETCHING DATA FOR: UIM
FETCHING DATA FOR: ENO
FETCHING DATA FOR: KHI
TRAIN NUMBER 169, 0.22533333333333333
TRAIN NUMBER 170, 0.22666666666666666
TRAIN NUMBER

TRAIN NUMBER 320, 0.4266666666666667
TRAIN NUMBER 321, 0.428
TRAIN NUMBER 322, 0.42933333333333334
TRAIN NUMBER 323, 0.43066666666666664
TRAIN NUMBER 324, 0.432
TRAIN NUMBER 325, 0.43333333333333335
TRAIN NUMBER 326, 0.43466666666666665
TRAIN NUMBER 327, 0.436
FETCHING DATA FOR: MN
TRAIN NUMBER 328, 0.43733333333333335
TRAIN NUMBER 329, 0.43866666666666665
TRAIN NUMBER 330, 0.44
TRAIN NUMBER 331, 0.44133333333333336
TRAIN NUMBER 332, 0.44266666666666665
TRAIN NUMBER 333, 0.444
TRAIN NUMBER 334, 0.44533333333333336
TRAIN NUMBER 335, 0.44666666666666666
FETCHING DATA FOR: YKST
FETCHING DATA FOR: YKSV
TRAIN NUMBER 336, 0.448
FETCHING DATA FOR: PYK
TRAIN NUMBER 337, 0.4493333333333333
TRAIN NUMBER 338, 0.45066666666666666
TRAIN NUMBER 339, 0.452
TRAIN NUMBER 340, 0.4533333333333333
FETCHING DATA FOR: TLV
TRAIN NUMBER 341, 0.45466666666666666
TRAIN NUMBER 342, 0.456
TRAIN NUMBER 343, 0.4573333333333333
TRAIN NUMBER 344, 0.45866666666666667
TRAIN NUMBER 345, 0.46
TRAIN NUMBER 346, 0.46133333

...

TRAIN NUMBER 676, 0.9013333333333333
TRAIN NUMBER 677, 0.9026666666666666
TRAIN NUMBER 678, 0.904
TRAIN NUMBER 679, 0.9053333333333333
TRAIN NUMBER 680, 0.9066666666666666
TRAIN NUMBER 681, 0.908
TRAIN NUMBER 682, 0.9093333333333333
TRAIN NUMBER 683, 0.9106666666666666
TRAIN NUMBER 684, 0.912
TRAIN NUMBER 685, 0.9133333333333333
TRAIN NUMBER 686, 0.9146666666666666
TRAIN NUMBER 687, 0.916
TRAIN NUMBER 688, 0.9173333333333333
TRAIN NUMBER 689, 0.9186666666666666
TRAIN NUMBER 690, 0.92
TRAIN NUMBER 691, 0.9213333333333333
TRAIN NUMBER 692, 0.9226666666666666
TRAIN NUMBER 693, 0.924
TRAIN NUMBER 694, 0.9253333333333333
TRAIN NUMBER 695, 0.9266666666666666
TRAIN NUMBER 696, 0.928
TRAIN NUMBER 697, 0.9293333333333333
TRAIN NUMBER 698, 0.9306666666666666
TRAIN NUMBER 699, 0.932
TRAIN NUMBER 700, 0.9333333333333333
TRAIN NUMBER 701, 0.9346666666666666
TRAIN NUMBER 702, 0.936
TRAIN NUMBER 703, 0.9373333333333334
TRAIN NUMBER 704, 0.9386666666666666
TRAIN NUMBER 705, 0.94
TRAIN NUMBER 706, 0.94

FETCHING DATA FOR: MRK
FETCHING DATA FOR: HPS
FETCHING DATA FOR: PM
FETCHING DATA FOR: LOL
FETCHING DATA FOR: HAU
FETCHING DATA FOR: KSA
FETCHING DATA FOR: HIR
FETCHING DATA FOR: MI
FETCHING DATA FOR: OT
FETCHING DATA FOR: LKL
FETCHING DATA FOR: MYT
FETCHING DATA FOR: MR
FETCHING DATA FOR: KII
FETCHING DATA FOR: HLS
FETCHING DATA FOR: VHJ
FETCHING DATA FOR: SPÄ
FETCHING DATA FOR: HJ
TRAIN NUMBER 43, 0.05733333333333333
TRAIN NUMBER 44, 0.058666666666666666
TRAIN NUMBER 45, 0.06
FETCHING DATA FOR: KAO
FETCHING DATA FOR: KON
FETCHING DATA FOR: PTO
FETCHING DATA FOR: MLL
FETCHING DATA FOR: KVJ
FETCHING DATA FOR: LMP
FETCHING DATA FOR: VAA
FETCHING DATA FOR: NSK
FETCHING DATA FOR: UTJ
FETCHING DATA FOR: HYR
FETCHING DATA FOR: MH
FETCHING DATA FOR: PKL
TRAIN NUMBER 46, 0.06133333333333333
TRAIN NUMBER 47, 0.06266666666666666
TRAIN NUMBER 48, 0.064
TRAIN NUMBER 49, 0.06533333333333333
TRAIN NUMBER 50, 0.06666666666666667
TRAIN NUMBER 51, 0.068
TRAIN NUMBER 52, 0.06933333333333333
FETCHING DA

FETCHING DATA FOR: KHK
FETCHING DATA FOR: HPL
FETCHING DATA FOR: VMO
FETCHING DATA FOR: PJM
FETCHING DATA FOR: MÄK
FETCHING DATA FOR: LPV
FETCHING DATA FOR: KIL
FETCHING DATA FOR: KEA
FETCHING DATA FOR: KNI
FETCHING DATA FOR: KVH
FETCHING DATA FOR: TRL
FETCHING DATA FOR: EPO
FETCHING DATA FOR: KLH
FETCHING DATA FOR: MAS
FETCHING DATA FOR: VKH
FETCHING DATA FOR: JRS
FETCHING DATA FOR: HEK
FETCHING DATA FOR: TOL
FETCHING DATA FOR: KKN
FETCHING DATA FOR: STI
FETCHING DATA FOR: IKO
FETCHING DATA FOR: PKU
FETCHING DATA FOR: ERV
FETCHING DATA FOR: SLO
FETCHING DATA FOR: PO
FETCHING DATA FOR: PIK
FETCHING DATA FOR: KUT
TRAIN NUMBER 196, 0.2613333333333333
TRAIN NUMBER 197, 0.26266666666666666
TRAIN NUMBER 198, 0.264
TRAIN NUMBER 199, 0.2653333333333333
TRAIN NUMBER 200, 0.26666666666666666
TRAIN NUMBER 201, 0.268
TRAIN NUMBER 202, 0.2693333333333333
TRAIN NUMBER 203, 0.27066666666666667
TRAIN NUMBER 204, 0.272
TRAIN NUMBER 205, 0.2733333333333333
TRAIN NUMBER 206, 0.27466666666666667
TRAIN NU

FETCHING DATA FOR: OLT
FETCHING DATA FOR: NOK
TRAIN NUMBER 369, 0.492
FETCHING DATA FOR: RAT
FETCHING DATA FOR: RHE
FETCHING DATA FOR: R530
TRAIN NUMBER 370, 0.49333333333333335
TRAIN NUMBER 371, 0.49466666666666664
TRAIN NUMBER 372, 0.496
TRAIN NUMBER 373, 0.49733333333333335
TRAIN NUMBER 374, 0.49866666666666665
TRAIN NUMBER 375, 0.5
FETCHING DATA FOR: TOR
TRAIN NUMBER 376, 0.5013333333333333
TRAIN NUMBER 377, 0.5026666666666667
TRAIN NUMBER 378, 0.504
TRAIN NUMBER 379, 0.5053333333333333
TRAIN NUMBER 380, 0.5066666666666667
FETCHING DATA FOR: R934
FETCHING DATA FOR: ARO
FETCHING DATA FOR: YPY
FETCHING DATA FOR: VUS
TRAIN NUMBER 381, 0.508
TRAIN NUMBER 382, 0.5093333333333333
FETCHING DATA FOR: TLV
FETCHING DATA FOR: YKST
FETCHING DATA FOR: YKSV
TRAIN NUMBER 383, 0.5106666666666667
TRAIN NUMBER 384, 0.512
TRAIN NUMBER 385, 0.5133333333333333
FETCHING DATA FOR: VLM
FETCHING DATA FOR: MLK
FETCHING DATA FOR: VKT
TRAIN NUMBER 386, 0.5146666666666667
TRAIN NUMBER 387, 0.516
TRAIN NUMBER 3

TRAIN NUMBER 611, 0.8146666666666667
TRAIN NUMBER 612, 0.816
TRAIN NUMBER 613, 0.8173333333333334
TRAIN NUMBER 614, 0.8186666666666667
TRAIN NUMBER 615, 0.82
TRAIN NUMBER 616, 0.8213333333333334
TRAIN NUMBER 617, 0.8226666666666667
TRAIN NUMBER 618, 0.824
TRAIN NUMBER 619, 0.8253333333333334
TRAIN NUMBER 620, 0.8266666666666667
TRAIN NUMBER 621, 0.828
TRAIN NUMBER 622, 0.8293333333333334
TRAIN NUMBER 623, 0.8306666666666667
TRAIN NUMBER 624, 0.832
TRAIN NUMBER 625, 0.8333333333333334
TRAIN NUMBER 626, 0.8346666666666667
TRAIN NUMBER 627, 0.836
TRAIN NUMBER 628, 0.8373333333333334
TRAIN NUMBER 629, 0.8386666666666667
TRAIN NUMBER 630, 0.84
TRAIN NUMBER 631, 0.8413333333333334
TRAIN NUMBER 632, 0.8426666666666667
TRAIN NUMBER 633, 0.844
TRAIN NUMBER 634, 0.8453333333333334
TRAIN NUMBER 635, 0.8466666666666667
TRAIN NUMBER 636, 0.848
TRAIN NUMBER 637, 0.8493333333333334
TRAIN NUMBER 638, 0.8506666666666667
TRAIN NUMBER 639, 0.852
TRAIN NUMBER 640, 0.8533333333333334
TRAIN NUMBER 641, 0.85

In [12]:
from sklearn import linear_model, model_selection, pipeline, preprocessing

best_score = 0
best_reg = linear_model.SGDClassifier(max_iter=1000)

for i in range(300):
    print(i)
    training_data, test_data, train_target, test_target = model_selection.train_test_split(combined_train_data, combined_late_amounts, train_size=0.8)
    train_target = np.ravel(train_target)
    test_target = np.ravel(test_target)

    reg = linear_model.SGDClassifier(max_iter=1000)
    reg.fit(training_data, train_target)
    reg_score = reg.score(test_data, test_target)
    if reg_score > best_score:
        best_score = reg_score
        best_reg = reg
        print("NEW BEST SCORE: " + str(best_score))
        
print(best_reg.score(test_data, test_target))

0
NEW BEST SCORE: 0.7566666666666667
1
2
3
4
NEW BEST SCORE: 0.7766666666666666
5
6
7
8
NEW BEST SCORE: 0.7866666666666666
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
NEW BEST SCORE: 0.7877777777777778
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
NEW BEST SCORE: 0.7905555555555556
49
50
51
52
53
NEW BEST SCORE: 0.7994444444444444
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
NEW BEST SCORE: 0.8
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219


In [29]:
from sklearn.impute import SimpleImputer


def extract_time_tables_station(train_stops_):
    train_stops = pd.json_normalize(train_stops_)

    train_stops.apply(get_weather, axis=1)

    for stop in train_stops['station.shortCode'].unique():
        if type(stop) == str:
            if stop not in df.columns:
                df[stop] = np.nan

            df[stop][train_stops_.name] = weather_data[stop]
            

stations_to_check = ['HKI', 'PSL', 'TKL', 'KE'] # List here stations that are checked, and possible tweeted about
results = []


for station in stations_to_check:
    date = datetime.date.today()
    unix_time = time.mktime(date.timetuple())
    
    q = (
        """ 
        {
        """
          f'  trainsByStationAndQuantity(station: "{station}",'
        """
        """
          f'  take: {200},'
        """
            where: {
                and: [
                    {or: [{deleted: {unequals: null}}, 
                    {deleted: {equals: false}}]}, 
                    {cancelled: {equals: false}}, 
                    {operator: {shortCode: {equals: "vr"}}}
                    ]  
            }
          ) 
          {
            trainNumber
            departureDate
            timeTableRows {
              station {
                shortCode
                location
              }
            }
          }
        }
        """
        )
    
    result = fetch_train_data(q)
    recs = result['data']['trainsByStationAndQuantity']

    df = pd.json_normalize(recs)
    all_stops = pd.json_normalize(df['timeTableRows'])
    df = df.drop(['timeTableRows'], axis=1)
    
    weather_data = open_weather_data(date)
    all_stops.apply(extract_time_tables_station, axis=1)
    save_weather_data(date, weather_data)
    
    df['departureDate'] = date.month
    df.fillna(0, inplace=True)
    
    for station in best_reg.feature_names_in_:
        if station not in df.columns:
            df[station] = 0.0
    
    how_many_late = 0
    for prediction in best_reg.predict(df):
        if prediction == 1:
            how_many_late = how_many_late + 1
            
    if how_many_late >= df.shape[0] / 2:
        results.append(True)
    else:
        results.append(False)
        
print(stations_to_check)
print(results)

/tmp/ipykernel_25422/1017715163.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[station] = 0.0
/home/otters/git/VRbad.io/venv/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/tmp/ipykernel_25422/1017715163.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[station] = 0.0
/home/otters/git

['HKI', 'PSL', 'TKL', 'KE']
[False, False, True, True]


/tmp/ipykernel_25422/1017715163.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[station] = 0.0
/home/otters/git/VRbad.io/venv/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
